markdown


In [43]:
#Imports
import yfinance as yf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import math
from pathlib import Path

In [44]:
eurusd = yf.download('EURUSD=X', 
                     start='2023-11-16',  # adjust date as needed
                     end='2024-11-16',    # adjust date as needed
                     interval='1h')

[*********************100%%**********************]  1 of 1 completed


In [45]:
# Display first few rows
display(eurusd.head())

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2023-11-16 00:00:00+00:00,1.085658,1.085894,1.085423,1.085894,1.085894,0
2023-11-16 01:00:00+00:00,1.085658,1.085658,1.084599,1.084716,1.084716,0
2023-11-16 02:00:00+00:00,1.084599,1.084599,1.083306,1.083658,1.083658,0
2023-11-16 03:00:00+00:00,1.083541,1.083893,1.083541,1.083541,1.083541,0
2023-11-16 04:00:00+00:00,1.083541,1.084011,1.083541,1.084011,1.084011,0


In [46]:
# Basic info about the data
print("\nDataset Info:")
print(eurusd.info())


Dataset Info:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6205 entries, 2023-11-16 00:00:00+00:00 to 2024-11-15 22:00:00+00:00
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       6205 non-null   float64
 1   High       6205 non-null   float64
 2   Low        6205 non-null   float64
 3   Close      6205 non-null   float64
 4   Adj Close  6205 non-null   float64
 5   Volume     6205 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 339.3 KB
None


In [47]:


# Calculate 10-day moving average
eurusd['10DayMAvg'] = eurusd['Close'].rolling(window=240).mean()

# Add previous day's close
eurusd['PrevClose'] = eurusd['Close'].shift(24)  # Shift by 24 periods for previous day's close

# Create condition column
eurusd['Condition'] = (
    (eurusd['Open'] < eurusd['PrevClose']) & 
    (eurusd['Open'] < eurusd['10DayMAvg'])
).astype(int)  # Convert True/False to 1/0


In [48]:
display(eurusd)

,Open,High,Low,Close,Adj Close,Volume,10DayMAvg,PrevClose,Condition
Datetime,,,,,,,,,
2023-11-16 00:00:00+00:00,1.085658,1.085894,1.085423,1.085894,1.085894,0,NaN,NaN,0
2023-11-16 01:00:00+00:00,1.085658,1.085658,1.084599,1.084716,1.084716,0,NaN,NaN,0
2023-11-16 02:00:00+00:00,1.084599,1.084599,1.083306,1.083658,1.083658,0,NaN,NaN,0
2023-11-16 03:00:00+00:00,1.083541,1.083893,1.083541,1.083541,1.083541,0,NaN,NaN,0
2023-11-16 04:00:00+00:00,1.083541,1.084011,1.083541,1.084011,1.084011,0,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...
2024-11-15 18:00:00+00:00,1.055186,1.055743,1.054185,1.055409,1.055409,0,1.071535,1.055075,0
2024-11-15 19:00:00+00:00,1.055298,1.055409,1.053963,1.054185,1.054185,0,1.071412,1.055075,0
2024-11-15 20:00:00+00:00,1.053963,1.053963,1.052078,1.052964,1.052964,0,1.071281,1.052299,0


In [49]:

# Count occurrences
total_occurrences = eurusd['Condition'].sum()

# Count total trading days
total_days = len(set(eurusd.index.strftime('%Y-%m-%d')))

# Print results
print("\nAnalysis Results:")
print(f"Total Trading Days: {total_days}")
print(f"Times Condition Met: {total_occurrences}")
print(f"Average Times Per Day: {(total_occurrences/total_days):.2f}")
print(f"Percentage of Hours: {(total_occurrences/len(eurusd)*100):.2f}%")

# Show days where condition was met
condition_met = eurusd[eurusd['Condition'] == 1]
print("\nLast 5 occurrences where condition was met:")
print(condition_met[['Open', 'PrevClose', '10DayMAvg', 'Condition']].tail())

# Group by date to see daily occurrences
daily_counts = eurusd.groupby(eurusd.index.strftime('%Y-%m-%d'))['Condition'].sum()
print("\nDaily Breakdown (last 5 days):")
print(daily_counts.tail())

# Day of week analysis (0=Monday, 6=Sunday)
day_of_week_counts = eurusd.groupby(eurusd.index.dayofweek)['Condition'].sum()
print("\nDay of Week Breakdown:")
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
for day, count in day_of_week_counts.items():
    print(f"{days[day]}: {count}")

# Optional: Save to CSV
#eurusd.to_csv('eurusd_analysis.csv')#


Analysis Results:
Total Trading Days: 262
Times Condition Met: 2057
Average Times Per Day: 7.85
Percentage of Hours: 33.15%

Last 5 occurrences where condition was met:
                               Open  PrevClose  10DayMAvg  Condition
Datetime                                                            
2024-11-15 06:00:00+00:00  1.054185   1.054741   1.073069          1
2024-11-15 15:00:00+00:00  1.053297   1.057865   1.071913          1
2024-11-15 16:00:00+00:00  1.053741   1.056412   1.071786          1
2024-11-15 17:00:00+00:00  1.055632   1.056189   1.071661          1
2024-11-15 21:00:00+00:00  1.052853   1.053186   1.071158          1

Daily Breakdown (last 5 days):
Datetime
2024-11-11    24
2024-11-12    24
2024-11-13    20
2024-11-14    22
2024-11-15    11
Name: Condition, dtype: int32

Day of Week Breakdown:
Monday: 383
Tuesday: 447
Wednesday: 444
Thursday: 392
Friday: 391


In [50]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Download data
end_date = datetime.now()
start_date = end_date - timedelta(days=365)

# Get hourly data
eurusd = yf.download('EURUSD=X', 
                    start=start_date,
                    end=end_date,
                    interval='1h')

# Calculate 10-day moving average
eurusd['10DayMAvg'] = eurusd['Close'].rolling(window=240).mean()

# Add previous day's close
eurusd['PrevClose'] = eurusd['Close'].shift(24)

# Create condition column
eurusd['Condition'] = (
   (eurusd['Open'] < eurusd['PrevClose']) & 
   (eurusd['Open'] < eurusd['10DayMAvg'])
).astype(int)

# Initial balance
initial_balance = 10000
position_size = 10000

# Calculate PNL for each trade
eurusd['PNL'] = np.where(
   eurusd['Condition'] == 1,
   (eurusd['Open'] - eurusd['Close']) * position_size,
   0
)

# Calculate Returns as percentage
eurusd['Returns'] = np.where(
   eurusd['Condition'] == 1,
   (eurusd['Open'] - eurusd['Close']) / eurusd['Open'] * 100,
   0
)

# Calculate running balance
eurusd['Balance'] = initial_balance + eurusd['PNL'].cumsum()

# Print results
print("\nTrading Account Performance:")
print(f"Initial Balance: ${initial_balance:,.2f}")
print(f"Final Balance: ${eurusd['Balance'].iloc[-1]:,.2f}")
print(f"Total Profit/Loss: ${eurusd['Balance'].iloc[-1] - initial_balance:,.2f}")
print(f"Return on Initial Capital: {((eurusd['Balance'].iloc[-1] - initial_balance)/initial_balance*100):,.2f}%")

# Calculate drawdown
eurusd['Peak'] = eurusd['Balance'].cummax()
eurusd['Drawdown'] = (eurusd['Balance'] - eurusd['Peak']) / eurusd['Peak'] * 100

# Print risk metrics
print("\nRisk Metrics:")
print(f"Maximum Balance: ${eurusd['Balance'].max():,.2f}")
print(f"Maximum Drawdown: {eurusd['Drawdown'].min():,.2f}%")
print(f"Average Daily Return: {eurusd['Returns'].mean():,.2f}%")

# Show daily balance changes
daily_balance = eurusd.groupby(eurusd.index.strftime('%Y-%m-%d'))['Balance'].last()
print("\nLast 5 Days Closing Balance:")
print(daily_balance.tail())

# Show worst drawdown periods
print("\nWorst Drawdown Periods:")
worst_drawdowns = eurusd.nsmallest(5, 'Drawdown')[['Balance', 'Peak', 'Drawdown']]
print(worst_drawdowns)

# Calculate monthly returns
monthly_returns = eurusd.groupby(eurusd.index.strftime('%Y-%m'))[['PNL']].sum()
monthly_returns['Monthly_Return'] = monthly_returns['PNL'] / initial_balance * 100
print("\nMonthly Returns:")
print(monthly_returns['Monthly_Return'].tail())

# Optional: Save to CSV
eurusd.to_csv('eurusd_balance_sheet.csv')

# Optional: Basic statistics about the balance
print("\nBalance Statistics:")
balance_stats = eurusd['Balance'].describe()
print(balance_stats)

[*********************100%%**********************]  1 of 1 completed


Trading Account Performance:
Initial Balance: $10,000.00
Final Balance: $10,436.47
Total Profit/Loss: $436.47
Return on Initial Capital: 4.36%

Risk Metrics:
Maximum Balance: $10,534.30
Maximum Drawdown: -2.42%
Average Daily Return: 0.00%

Last 5 Days Closing Balance:
Datetime
2024-11-11    10457.413197
2024-11-12    10483.496189
2024-11-13    10482.081175
2024-11-14    10493.135452
2024-11-15    10436.472893
Name: Balance, dtype: float64

Worst Drawdown Periods:
                               Balance          Peak  Drawdown
Datetime                                                      
2024-03-20 17:00:00+00:00  9952.971935  10200.128555 -2.423074
2024-03-20 18:00:00+00:00  9952.971935  10200.128555 -2.423074
2024-03-20 19:00:00+00:00  9952.971935  10200.128555 -2.423074
2024-03-20 20:00:00+00:00  9952.971935  10200.128555 -2.423074
2024-03-20 21:00:00+00:00  9952.971935  10200.128555 -2.423074

Monthly Returns:
Datetime
2024-07   -0.757396
2024-08    0.185537
2024-09   -0.528920
202